### Homework 5: Question search engine



Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.



__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [2]:
%pip install --upgrade transformers datasets accelerate deepspeed

import torch

import torch.nn as nn

import torch.nn.functional as F

import transformers

import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 91.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.2 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.15.2-py3-none-any.whl size=1521851 sha256=22723f58de9a7192b0df38428214e934f7c7390aa5a0c24e8918f03da10a43e2
  Stored in directory: /root/.cache/pip/wheels/53/f1/46/5d54ff3c9f0af8cab280e54887af31c3a6b489b409ba8e3f5c
Successfully built deepspeed
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting uninstall: transformers
    Found existing installation: transfor

### Load data and model

In [4]:
qqp = datasets.load_dataset('SetFit/qqp')

print('\n')

print("Sample[0]:", qqp['train'][0])

print("Sample[3]:", qqp['train'][3])

README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/70.8M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/7.83M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/76.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [33]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name).to(0)

### Tokenize the data

In [100]:
MAX_LENGTH = 128

def preprocess_function(examples):

    result = tokenizer(

        examples['text1'], examples['text2'],

        padding='max_length', max_length=MAX_LENGTH, truncation=True

    )

    result['label'] = examples['label']

    return result



qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

In [5]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 point)



We randomly chose a model trained on QQP - but is it any good?



One way to measure this is with validation accuracy - which is what you will implement next.



Here's the interface to help you do that:

In [56]:
val_set = qqp_preprocessed['validation']

val_loader = torch.utils.data.DataLoader(

    val_set, batch_size=4, shuffle=False, collate_fn=transformers.default_data_collator

)

In [82]:
for batch in val_loader:
        
     break  # here be your training code

print("Sample batch:", batch)



with torch.no_grad():

  predicted = model(

      input_ids=batch['input_ids'],

      attention_mask=batch['attention_mask'],

      token_type_ids=batch['token_type_ids']

  )



#print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())
print(batch)

Sample batch: {'labels': tensor([0, 0, 1, 0]), 'idx': tensor([0, 1, 2, 3]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

__Your task__ is to measure the validation accuracy of your model.

Doing so naively may take several hours. Please make sure you use the following optimizations:



- run the model on GPU with no_grad

- using batch size larger than 1

- use optimize data loader with num_workers > 1

- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
probs = F.softmax(predicted.logits)
pred_labels = torch.argmax(probs, dim = 1)
correct = torch.eq(pred_labels, batch['labels'])
batch_accuracy = correct.float().mean()
accuracy.append(batch_accuracy)
clear_output(wait=True)

In [66]:
from tqdm import tqdm
from IPython.display import clear_output


model.eval()
with torch.no_grad():
    # Initialize accuracy counter
    total_correct = 0
    total_samples = 0

    # Iterate over validation data loader
    for batch in tqdm(val_loader):
        # Move batch to device
        batch = {key: value.to(0) for key, value in batch.items()}
        predicted = model(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],  
          token_type_ids=batch['token_type_ids']
        )
        _, predicted = torch.max(predicted.logits, dim=1)

        total_correct += (predicted == batch["labels"]).sum().item()
        total_samples += batch["labels"].size(0)
    accuracy = total_correct / total_samples
    print(accuracy)
#accuracy = <Validation accuracy, between 0 and 1>


100%|██████████| 10108/10108 [03:10<00:00, 53.07it/s]

0.9083848627256987


In [67]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (4 points)



For this task, you have two options:



__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).



You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.





__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [3]:
debert = 'microsoft/deberta-v3-base'

debert_tokenizer = transformers.AutoTokenizer.from_pretrained(debert)

debert_model = transformers.AutoModelForSequenceClassification.from_pretrained(debert).to(0)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
MAX_LENGTH = 128

def preprocess_function(examples):

    result = debert_tokenizer(

        examples['text1'], examples['text2'],

        padding='max_length', max_length=MAX_LENGTH, truncation=True

    )

    result['label'] = examples['label']

    return result



debert_qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [7]:
train_set = debert_qqp_preprocessed['train']
val_set = debert_qqp_preprocessed['validation']
test_set = debert_qqp_preprocessed['test']

train_loader = torch.utils.data.DataLoader(

    train_set, batch_size=16, shuffle=True, collate_fn=transformers.default_data_collator

)
val_loader = torch.utils.data.DataLoader(

    val_set, batch_size=16, shuffle=False, collate_fn=transformers.default_data_collator

)
test_loader = torch.utils.data.DataLoader(

    test_set, batch_size=16, shuffle=False, collate_fn=transformers.default_data_collator

)

In [16]:
from tqdm import tqdm
from IPython.display import clear_output


debert_model.eval()
with torch.no_grad():
    # Initialize accuracy counter
    total_correct = 0
    total_samples = 0

    # Iterate over validation data loader
    for batch in tqdm(val_loader):
        # Move batch to device
        batch = {key: value.to(0) for key, value in batch.items()}
        predicted = debert_model(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],  
          token_type_ids=batch['token_type_ids']
        )
        _, predicted = torch.max(predicted.logits, dim=1)

        total_correct += (predicted == batch["labels"]).sum().item()
        total_samples += batch["labels"].size(0)
    accuracy = total_correct / total_samples
    print(accuracy)
#accuracy = <Validation accuracy, between 0 and 1>


100%|██████████| 2527/2527 [03:28<00:00, 12.09it/s]

0.6302745486025229


Начальная точность модели состовляет **0.6302745486025229**


In [9]:
from tqdm import tqdm

In [13]:
optimizer = torch.optim.Adam(debert_model.parameters(), lr=5e-7)
loss_fn = nn.CrossEntropyLoss()

# Train loop
for epoch in range(1):
    debert_model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(0)
        attention_mask = batch['attention_mask'].to(0)
        labels = batch['labels'].to(0)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = debert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = loss_fn(outputs.logits, labels)

        # Backward pass
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Accumulate loss
        total_loss += loss.item()

    # Print training loss
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')


 77%|███████▋  | 17470/22741 [1:19:53<24:06,  3.64it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 22741/22741 [1:43:58<00:00,  3.65it/s]


Epoch 1, Loss: 0.3244434038593138


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not SequenceClassifierOutput

In [17]:
from IPython.display import clear_output


debert_model.eval()
with torch.no_grad():
    # Initialize accuracy counter
    total_correct = 0
    total_samples = 0

    # Iterate over validation data loader
    for batch in tqdm(val_loader):
        # Move batch to device
        batch = {key: value.to(0) for key, value in batch.items()}
        predicted = debert_model(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],  
          token_type_ids=batch['token_type_ids']
        )
        _, predicted = torch.max(predicted.logits, dim=1)

        total_correct += (predicted == batch["labels"]).sum().item()
        total_samples += batch["labels"].size(0)
    accuracy = total_correct / total_samples
    print(accuracy)
#accuracy = <Validation accuracy, between 0 and 1>


100%|██████████| 2527/2527 [03:28<00:00, 12.12it/s]

0.8791986148899332


In [ ]:
from tqdm import tqdm
from IPython.display import clear_output


debert_model.eval()
with torch.no_grad():
    # Initialize accuracy counter
    total_correct = 0
    total_samples = 0

    # Iterate over validation data loader
    for batch in tqdm(test_loader):
        # Move batch to device
        batch = {key: value.to(0) for key, value in batch.items()}
        predicted = debert_model(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],  
          token_type_ids=batch['token_type_ids']
        )
        _, predicted = torch.max(predicted.logits, dim=1)

        total_correct += (predicted == batch["labels"]).sum().item()
        total_samples += batch["labels"].size(0)
    accuracy = total_correct / total_samples
    print(accuracy)
#accuracy = <Validation accuracy, between 0 and 1>


In [ ]:
PATH = 'debert_v1.pt'

In [ ]:
torch.save(debert_model.state_dict(), PATH)

### Task 3: try the full pipeline (1 point)



Finally, it is time to use your model to find duplicate questions.

Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.



Showcase how your function works with at least 5 examples.

In [15]:
val_set

Dataset({
    features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 40430
})

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.